In [4]:
# change these to try this notebook out# chang 
BUCKET = 'atos-asl'
PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1'
REGION = 'us-central1'

In [34]:
%%bash
pwd
cd ../../
pwd
git status


/content/datalab/energy_forcasing/bram/explore
/content/datalab/energy_forcasing
On branch master
Your branch is up-to-date with 'origin/master'.
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	bram/

nothing added to commit but untracked files present (use "git add" to track)


In [5]:
%%bash
lshw - List Hardware

bash: line 1: lshw: command not found


In [6]:
import os
import google.datalab.bigquery as bq
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [7]:
from pandas.plotting import scatter_matrix

In [8]:
market_query = """
select 
 *
from energy.historical_forecasting 
"""
df = bq.Query(market_query).execute().result().to_dataframe()
print(df.shape)
df.head(10)

(272592, 12)


point     prediction_date  wind_speed_100m  wind_direction_100m  \
0      5 2015-05-13 12:00:00              8.9                236.7   
1      5 2015-07-03 11:00:00              9.6                185.1   
2      5 2015-07-03 12:00:00             10.0                187.1   
3      5 2015-07-03 13:00:00             10.0                192.5   
4      5 2015-07-03 14:00:00             10.0                197.8   
5      5 2015-07-03 15:00:00             10.0                203.1   
6      5 2015-07-10 09:00:00              0.6                211.0   
7      5 2015-08-27 11:00:00              9.3                213.3   
8      5 2015-08-27 12:00:00              8.0                214.5   
9      5 2015-08-27 13:00:00              8.2                217.7   

   temperature  air_density  pressure  precipitation  wind_gust  radiation  \
0         23.7          1.0     854.7            0.0       13.8      603.0   
1         23.4          1.0     852.7            0.0       11.2      643.6   
2         24.4          1.0     852.8            0.0       10.7      638.0   
3         24.6          1.0     852.8            0.0       10.7      746.4   
4         24.8          1.0     852.7            0.0       10.7      868.3   
5         25.0          1.0     852.7            0.0       10.7      960.0   
6         25.2          1.0     852.4            0.0        0.9      410.0   
7         22.4          1.0     848.0            0.0       13.5      451.3   
8         22.6          1.0     848.1            0.0       12.4      287.0   
9         22.8          1.0     848.2            0.0       12.3      509.1   

   wind_speed  wind_direction  
0         6.1           237.2  
1         7.0           184.1  
2         7.4           186.7  
3         7.4           192.3  
4         7.3           197.8  
5         7.3           203.4  
6         0.3           220.8  
7         7.1           211.6  
8         6.3           213.0  
9         6.5           216.6

In [9]:
df.describe()

point  wind_speed_100m  wind_direction_100m    temperature  \
count  272592.000000    272592.000000        272592.000000  272592.000000   
mean        9.500000         4.637073           194.652307      14.483588   
std         5.188137         2.916167           102.063648      10.434206   
min         1.000000         0.000000             0.000000     -24.700000   
25%         5.000000         2.500000           109.700000       6.900000   
50%         9.500000         4.000000           209.800000      13.400000   
75%        14.000000         6.100000           277.800000      20.600000   
max        18.000000        29.100000           360.000000      57.600000   

         air_density       pressure  precipitation      wind_gust  \
count  272592.000000  272592.000000  272592.000000  272592.000000   
mean        1.135904     936.946597       0.124406       4.893374   
std         0.057938      47.494264       0.278393       3.639607   
min         0.960000     825.300000       0.000000       0.000000   
25%         1.090000     891.000000       0.000000       2.300000   
50%         1.130000     934.500000       0.000000       3.700000   
75%         1.180000     982.700000       0.100000       6.400000   
max         1.300000    1036.700000       2.100000      30.400000   

           radiation     wind_speed  wind_direction  
count  272592.000000  272592.000000   272592.000000  
mean      216.905158       3.146508      193.737157  
std       276.894263       1.980469      101.226174  
min         0.000000       0.000000        0.100000  
25%         0.000000       1.700000      111.575000  
50%        65.400000       2.700000      207.100000  
75%       398.600000       4.100000      275.600000  
max      1019.900000      21.500000      360.000000

In [10]:
mask = ['air_density', 'temperature', 'pressure', 'radiation', 'wind_speed_100m']

scatter_matrix(df[mask], figsize=(8, 8), diagonal='hist')

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f43e32fc5d0>,
      dtype=object)

/usr/local/envs/py2env/lib/python2.7/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [29]:
# calculate some distribution methods in the aggregate window
query_market_price = """
select 
  distinct(CAST(avg_mp.date as date)) as date,
  point,
  avg(avg_mp.avg_price) as avg_price,
  avg(wind_speed_100m) as avg_wind_speed_100m,
  avg(wind_direction_100m) as avg_wind_direction_100m,
  avg(temperature) as avg_temperature,
  avg(air_density) as avg_air_density,
  avg(pressure) as avg_pressure,
  avg(precipitation) as avg_precipitation,
  avg(wind_gust) as avg_wind_gust,
  avg(radiation) as avg_radiation,
  avg(wind_speed) as avg_wind_speed,
  avg(wind_direction) as avg_wind_direction,
  VARIANCE(wind_direction) as var_wind_direction,
  VARIANCE(wind_speed) as var_wind_speed,
  VARIANCE(radiation) as var_radiation,
  VARIANCE(wind_gust) as var_wind_gust,
  VARIANCE(pressure) as var_pressure,
  VARIANCE(wind_speed_100m) as var_wind_speed_100m,
  VARIANCE(air_density) as var_air_density,
  VARIANCE(temperature) as var_temperature,
  VARIANCE(precipitation) as var_precipitation,
  min(temperature) as min_temperature,
  max(temperature) as max_temperature,
  min(wind_speed) as min_wind_speed,
  max(wind_speed) as max_wind_speed,
  min(precipitation) as min_precipitation,
  max(precipitation) as max_precipitation
from energy.historical_forecasting as hf right join
  (SELECT
   CAST(mp.date as date) as date,
   avg(price) as avg_price
  FROM
    energy.market_price as mp
  WHERE mp.date > CAST('2015-01-01' AS TIMESTAMP) AND mp.date < CAST('2015-10-1' AS TIMESTAMP)
  group by CAST(mp.date as date)) as avg_mp
ON avg_mp.date = CAST(hf.prediction_date as date)
group by CAST(avg_mp.date as date), point
"""
df_market = bq.Query(query_market_price).execute().result().to_dataframe().dropna()
print(df_market.shape)
df_market.head(10)

(4860, 28)


date  point  avg_price  avg_wind_speed_100m  \
3   2015-07-24   10.0    58.7375             6.104167   
4   2015-07-24   12.0    58.7375             3.858333   
5   2015-07-24   11.0    58.7375             7.470833   
6   2015-07-24    5.0    58.7375             3.416667   
7   2015-07-24   14.0    58.7375             5.958333   
8   2015-07-24    6.0    58.7375             8.054167   
9   2015-07-24    8.0    58.7375             5.012500   
10  2015-07-24    2.0    58.7375             3.325000   
11  2015-07-24   18.0    58.7375             3.304167   
12  2015-07-24    9.0    58.7375             3.870833   

    avg_wind_direction_100m  avg_temperature  avg_air_density  avg_pressure  \
3                278.345833        22.025000         1.037500    877.791667   
4                308.983333        27.966667         1.027083    886.870833   
5                260.708333        31.645833         1.075000    940.100000   
6                215.037500        13.950000         1.031667    849.762500   
7                213.320833        24.591667         1.029167    878.958333   
8                272.804167        25.237500         1.081667    924.854167   
9                270.258333        30.483333         1.160417   1010.104167   
10               292.820833        15.195833         1.124583    930.304167   
11               269.995833        16.450000         1.057917    879.412500   
12               290.445833        18.537500         1.085833    908.162500   

    avg_precipitation  avg_wind_gust        ...          var_wind_speed_100m  \
3                 0.0       5.241667        ...                     4.495199   
4                 0.0       3.720833        ...                     3.162536   
5                 0.0       6.895833        ...                     2.693460   
6                 0.0       3.604167        ...                     1.212754   
7                 0.0       5.520833        ...                     6.316449   
8                 0.0       7.825000        ...                     1.126069   
9                 0.0       4.545833        ...                     4.197663   
10                0.0       2.958333        ...                     1.060217   
11                0.0       3.433333        ...                     1.720417   
12                0.0       4.350000        ...                     6.763895   

    var_air_density  var_temperature  var_precipitation  min_temperature  \
3          0.000654        53.824565                0.0             14.0   
4          0.000682        59.237101                0.0             18.7   
5          0.000974        77.833895                0.0             21.7   
6          0.000241        16.711304                0.0              8.0   
7          0.000773        55.923406                0.0             16.0   
8          0.001067        75.792880                0.0             15.6   
9          0.000865        56.762319                0.0             21.1   
10         0.000252        18.472591                0.0              9.2   
11         0.000313        21.036522                0.0              9.0   
12         0.000860        59.213750                0.0              9.1   

    max_temperature  min_wind_speed  max_wind_speed  min_precipitation  \
3              33.7             2.2             8.1                0.0   
4              39.6             0.6             4.9                0.0   
5              45.6             3.0             8.0                0.0   
6              21.0             0.7             4.4                0.0   
7              35.2             1.2             7.3                0.0   
8              38.5             3.9             7.6                0.0   
9              42.2             1.7             6.5                0.0   
10             22.4             0.9             3.3                0.0   
11             24.0             0.6             3.3                0.0   
12             31.3             0.5             6.1   

In [22]:
query_weather_station = """
SELECT
 *
FROM
  energy.station_location 
"""

In [23]:
query_historical_forecasting = """
SELECT
 count(*)
FROM
  energy.historical_forecasting
"""

In [29]:
# Call BigQuery and examine in dataframe
import google.datalab.bigquery as bq
df_market = bq.Query(query_market_price + " LIMIT 100").execute().result().to_dataframe()
df_market.head()
df_weather = bq.Query(query_weather_station + " LIMIT 100").execute().result().to_dataframe()
df_weather.head()
df_historical_forecasting = bq.Query(query_historical_forecasting + " LIMIT 100").execute().result().to_dataframe()
df_historical_forecasting.head()

f0_
0  272592

In [1]:
df_market.head()

NameError: name 'df_market' is not defined

In [27]:
df_historical_forecasting.head()

f0_
0  272592